In [1]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

from datetime import datetime, timedelta
from pandas import DataFrame, to_datetime, set_option
from TpsParse.Tps.TpsFile import TpsFile
import os
from numpy import quantile, nan
from math import ceil

set_option('display.max_columns', 500)
set_option('display.max_row', 1000)

from bokeh.resources import CDN
from bokeh.embed import components
from bokeh.models import Range1d, NumeralTickFormatter, ColumnDataSource, FactorRange
from bokeh.plotting import figure
from bokeh.transform import linear_cmap, factor_cmap
from bokeh.io import show, output_notebook
from bokeh.palettes import Category20_20

from django.db.models import Sum, DateField
from django.db.models.functions import Trunc
from Utils.clean_string import clean_string

output_notebook()

Loading BokehJS ...

In [2]:
from pprint import pprint

In [3]:
from app.models import Acumula, Imputaci

In [4]:
def parser(path):
    file = open(path, "r+b")
    tps = TpsFile(file)

    cols_and_type = []
    columns = []
    records = []
    for definition in tps.get_table_definitions():
        for field in definition.fields:
            column = field.field_name.split(':')[1].lower()
            col_and_type = (field.field_name.split(':')[1].lower(), field.type)
            if column not in columns:
                columns.append(column)
                cols_and_type.append(col_and_type)
        for record in tps.get_data_records(definition):
            records.append([r.strip().title() if isinstance(r, str) else r for r in record.values])

    table = []
    for record in records:
        dic = {}
        for col, rec in zip(columns, record):
            dic[col] = rec
        table.append(dic)

    df = DataFrame(table, columns=columns)

    name_list = ('fec', 'periodo')
    # Formatear la fecha.
    date_columns = [cname for cname, ctype in cols_and_type if cname.startswith(name_list) and ctype == 'SignedLong']
    for col in df[date_columns]:
        df[col] = datetime(1800, 12, 28) + df[col].map(timedelta)
        df.loc[df[col] == '1800-12-28', col] = None

    # Reemplazar los campos vacíos con None.
    df.replace(to_replace='', value=nan, inplace=True)
    df.replace(to_replace=0, value=None, inplace=True)
    
    # Borrar los registros que tengan todos valores nulos.
    df.dropna(axis='index', how='all', inplace=True)
    
    # Eliminar registros duplicados.
    df.drop_duplicates(keep='last', inplace=True)
    
    # Borrar las columnas que tengan todos valores nulos o ceros.
    df.dropna(axis='columns', how='all', inplace=True)
    df = df.loc[:, (df != 0).any(axis=0)]
    
    return df

In [8]:
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [17]:
imputaci = Imputaci.objects.values(
    'descripcion',
    'donde_acumula__descripcion',
    'total_egresos', )
#     .filter(
#     donde_acumula__descripcion='Insumos Tambo', 
#             total_egresos__gt=0
# )

df = DataFrame(imputaci)
df.rename(columns={'donde_acumula__descripcion': 'imputacion_acumulada'},
              inplace=True)
    
# gb_imputaci = DataFrame(df.groupby(
#     ['donde_acumula__descripcion','descripcion']).total_egresos.sum())
# gb_imputaci.sort_values('total_egresos', ascending=False, inplace=True)

In [22]:
df

,descripcion,imputacion_acumulada,total_egresos
0,Imp Ley 25413 Afip,Impuestos,0.00
1,Comision Transferencia,Impuestos,0.00
2,Franqueo,Impuestos,0.00
3,Comision Mant. De Cuenta,Impuestos,0.00
4,Comision Resumen,Impuestos,0.00
5,Comision Canje O/Bancos,Impuestos,0.00
6,Comision Gastos Chequera,Impuestos,0.00
7,Intereses Bancarios,Intereses,0.00
8,Semilla De Maiz,Insumos Agricultura,70695.00
9,Servicios De Saco De Grano,Servicios Agrop Agricultura,33862.80


In [5]:
from app.utils import get_imputaci
imputaci = get_imputaci()
df = DataFrame(imputaci)
df.rename(columns={'donde_acumula__descripcion': 'imputacion_acumulada'},
          inplace=True)

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

In [22]:
cols = {'compra_cta_cte': sum,
        'compra_contado': sum, 
        'nota_credito_recibida': sum,
        'nota_debito_recibida': sum, 
        'egresos_cta_cte': sum, 
        'egresos_contado': sum,
        'venta_cta_cte': sum, 
        'venta_contado': sum, 
        'nota_credito_emitida': sum,
        'nota_debito_emitida': sum, 
        'cobranzas': sum,
        'total_egresos': sum}
gb = DataFrame(df.groupby('imputacion_acumulada', as_index=False).agg(cols))

In [23]:
gb

,imputacion_acumulada,compra_cta_cte,compra_contado,nota_credito_recibida,nota_debito_recibida,egresos_cta_cte,egresos_contado,venta_cta_cte,venta_contado,nota_credito_emitida,nota_debito_emitida,cobranzas,total_egresos
0,Administracion,0.00,0.00,0.00,0.00,22394800.0,0.00,330440.0,0.0,0.0,0.0,22045700.0,0.00
1,Alquiler Rural,1845325.00,0.00,0.00,0.00,0.0,0.00,115996.0,0.0,0.0,0.0,0.0,1145077.00
2,Combustible,203688.64,1143.75,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,165549.75
3,Compra De Granos,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
4,Compra De Inmuebles,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
5,Compra Iva,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
6,Creditos,1368187.00,146074.00,0.00,0.00,0.0,425548.00,0.0,0.0,0.0,0.0,0.0,661811.80
7,Encomiendas,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
8,Ganaderia,99600.00,0.00,0.00,0.00,0.0,7000.00,0.0,0.0,0.0,0.0,0.0,30180.00
9,Gastos Varios,8787.65,6250.27,0.00,0.00,0.0,45407.60,0.0,0.0,0.0,0.0,0.0,15275.56


In [43]:
gb_imputaci

total_egresos
donde_acumula__descripcion  descripcion                                  
Alquiler Rural              Alquiler Campo Agricultura          184760.00
                            Alquiler Campo Ganaderia            324771.00
                            Alquiler Campo Tambo                635546.00
Combustible                 Aceites                              11450.70
                            Gas                                   2551.59
                            Gas Oil                             129562.00
                            Kerosene                               110.46
                            Nafta                                21875.00
Creditos                    Creditos                            615152.00
                            Intereses Creditos                   46659.80
Ganaderia                   Compra De Animales                   30180.00
Gastos Varios               Cuota Mutual Ramona                    140.00
                            Peaje                                  285.00
                            Varios                                3710.16
                            Viaticos Y Movilidad                 11140.40
Honorarios                  Honorarios Agronomicos               19423.10
                            Honorarios Contables                 21240.10
                            Honorarios Informaticos               7053.70
                            Honorarios Mandatario                 6596.31
                            Servicios Veterinarios                2440.00
Impuestos                   Dif. Alicuotas A.R.T.                 6059.72
                            Formulario 931                       69633.60
                            Ganancia Declaracion Jurada           2800.91
                            Renatea                                190.21
                            Tasa P/Hectaria Comite Cuenca        22743.70
                            Uatre                                 2410.28
                            Volante Pago Trab. De Casas            135.00
Indumentaria                Indumentaria Laboral                  1292.58
Insumos Agricultura         Agroquimicos                        356070.00
                            Bolsa Para Silo                      21571.20
                            Semilla Alfalfa                      16485.00
                            Semilla De Avena                     12919.80
                            Semilla De Maiz                      70695.00
                            Semilla De Sorgo                     14215.50
                            Semilla De Trigo                     11601.50
Insumos Ganaderia           Insumos Veterinarios Ganaderia        3809.02
                            Vacunacion                           15810.80
Insumos Para Cria           Pajuelas Inseminacion                22800.80
Insumos Tambo               Alquiler Vaca Para Tambo            164114.00
                            Balanceado                         1160490.00
                            Expeller De Soja                     35635.40
                            Insumos Limpieza Tambo               29895.40
                            Insumos Veterinarios Tambo          108967.00
                            Maiz Para Tambo                      44718.60
Intereses                   Intereses Por Mora                     784.21
Mantenimiento               Ferreteria                           32495.30
                            Mantenimiento Equipo De Frio         45919.30
                            Mantenimiento Ordeñadora             23378.60
                            Mantenimiento Rodados                24061.90
                            Mantenimientos Agropecuarios          3150.00
                            Mantenimientos Caminos               38100.00
                            Repuestos Y Reparaciones             92409.40
Publicidad                  Publicidad Grafica                  